In [38]:
import tensorflow as tf
from tensorboard.plugins import projector

import os, argparse
import pickle
import cv2, spacy, numpy as np
from keras.models import model_from_json
from keras.optimizers import SGD
import joblib
from keras import backend as K
import keras
import PIL
import pandas as pd

https://github.com/anujshah1003/Tensorboard-own-image-data-image-features-embedding-visualization

In [23]:
tf.__version__

'2.12.0'

In [34]:
PATH = os.getcwd()
LOG_DIR = PATH+ '/vis'
LOG_DIR

'/media1/data/claire/BearBackgroundsAndBehavior/vis'

In [13]:
data_path = '/media1/data/beardata/2021/2021_bodies/2021_singles_resized'
filenames = [os.path.join(data_path, os.listdir(data_path)[i]) for i in range(len(os.listdir(data_path)))]
filenames[0]

'/media1/data/beardata/2021/2021_bodies/2021_singles_resized/P1885008_0_Dune_bear.JPG'

## Create sprite.jpg file to visualize the images in the projector

In [16]:
images = [PIL.Image.open(filename) for filename in filenames]

image_width, image_height = images[0].size
one_square_size = int(np.ceil(np.sqrt(len(images))))
master_width = (image_width * one_square_size) 
master_height = image_height * one_square_size
spriteimage = PIL.Image.new(
    mode='RGBA',
    size=(master_width, master_height),
    color=(0,0,0,0))  # fully transparent
for count, image in enumerate(images):
    div, mod = divmod(count,one_square_size)
    h_loc = image_width*div
    w_loc = image_width*mod    
    spriteimage.paste(image,(w_loc,h_loc))
spriteimage.convert("RGB").save('sprite.jpg', transparency=0)

In [55]:
data = pd.read_csv('vis/embeddings_short2.tsv',sep='\t', header=None)
data

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.121979,0.180500,-0.101500,0.066033,-0.135600,-0.016117,0.075402,0.191959,0.250628,-0.363889,...,0.231436,-0.046597,0.026231,0.076395,-0.139318,-0.001033,0.289854,-0.176325,-0.153972,0.133803
1,-0.045447,0.044068,-0.137763,-0.144858,-0.040309,0.061621,-0.040065,0.259091,0.143567,-0.171977,...,0.220998,-0.062991,-0.006616,0.113453,-0.187876,0.015637,0.102037,-0.032357,-0.091988,0.161042
2,-0.068580,0.066230,-0.137090,0.043328,-0.002324,0.032082,0.117614,0.227936,0.248181,-0.240752,...,0.265619,-0.063197,0.069602,0.115273,-0.135968,0.081606,0.182556,0.024831,-0.085223,-0.010674
3,-0.050450,0.048493,-0.119378,-0.118063,-0.081385,0.035440,-0.018122,0.213397,0.057327,-0.107522,...,0.114653,-0.068044,0.105045,0.160640,-0.167509,-0.019187,0.135333,0.030994,-0.154920,0.148443
4,-0.124137,-0.001892,-0.145382,-0.035002,0.048863,0.067718,-0.101065,0.163431,0.102865,-0.279686,...,0.325686,-0.051851,0.155150,0.061030,-0.220549,-0.157861,0.067562,0.039090,-0.117150,-0.052039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.146987,0.122897,-0.145242,0.037112,-0.093457,0.066180,0.014781,0.236179,0.108328,-0.226237,...,0.168471,-0.066450,-0.035267,0.131283,-0.057009,0.103147,0.124321,-0.088344,-0.084050,0.113239
96,-0.063902,0.056132,-0.168397,-0.076362,-0.095062,0.071524,-0.003361,0.179502,0.171420,-0.247938,...,0.199202,-0.072698,0.039076,0.071906,-0.148656,0.065250,0.204031,-0.069838,-0.143941,0.108573
97,-0.072777,0.108234,-0.059540,-0.105778,-0.118174,0.034857,0.053566,0.183922,0.070087,-0.319679,...,0.137180,-0.062843,0.048731,0.131509,-0.080991,0.008009,0.103580,0.026899,-0.159456,0.141096
98,-0.053165,0.139670,-0.135295,-0.019297,0.032094,0.033921,-0.031600,0.269595,0.140264,-0.194193,...,0.233103,-0.055560,0.030744,0.121623,-0.271881,0.191935,0.096977,0.002296,-0.062845,0.101433


In [54]:
features = tf.Variable(data)
features

<tf.Variable 'Variable:0' shape=(100, 768) dtype=float64, numpy=
array([[-0.1219793 ,  0.18049994, -0.10149986, ..., -0.17632492,
        -0.15397169,  0.13380329],
       [-0.0454467 ,  0.04406803, -0.1377628 , ..., -0.0323573 ,
        -0.09198836,  0.16104163],
       [-0.06858011,  0.06622957, -0.13709007, ...,  0.02483103,
        -0.08522267, -0.01067392],
       ...,
       [-0.07277662,  0.10823404, -0.05953973, ...,  0.02689851,
        -0.15945576,  0.14109643],
       [-0.05316458,  0.13967015, -0.13529539, ...,  0.00229591,
        -0.06284451,  0.10143276],
       [-0.039361  , -0.05674601, -0.11724309, ..., -0.03087272,
        -0.2015423 ,  0.07577317]])>

In [56]:
with tf.compat.v1.Session() as sess:
    saver = tf.train.Saver([features])

    sess.run(features.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'images_4_classes.ckpt'))
    
    config = projector.ProjectorConfig()
    
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = features.name

    # Comment out if you don't want sprites
    embedding.sprite.image_path = os.path.join(LOG_DIR, 'sprite.jpg')
    
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)

2023-05-08 19:08:52.807908: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7308 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:02:00.0, compute capability: 7.5


AttributeError: module 'tensorflow._api.v2.train' has no attribute 'Saver'